# Step 0 - Set up the Cluster

## Overview
Our goal in this series of notebooks is to train a deep reinforcement learning autonomous driving model in a distributed way using a pool of virtual machines on Microsoft Azure. We will first go over the instructions for setting up a VM cluster to prepare for the training job. The details of the RL model and the training process will be covered in later notebooks. Please note that you will require an active [Azure subscription](https://azure.microsoft.com/en-us/free/) to run the code provided here.

## Create Azure service accounts

In this notebook, you will set up and provision a cluster of virtual machines which will be used to distribute the training job. Before we get started, please do the following:

   1. **Create an Azure storage account.** You will be using this account to create a file share that will be used by the cluster nodes to store the source code files. You can find the instructions to create the storage account [here](https://docs.microsoft.com/en-us/azure/storage/common/storage-create-storage-account). Please follow the instructions to create a general-purpose storage account and make a note of the ***resource group name***, ***account name*** and primary ***access key*** as you will be needing those shortly.
   
   2. **Create an Azure Batch account.** Azure Batch is a free Azure service that allows you to do cloud-scale job scheduling. You can find the instructions to create your batch account [here](https://docs.microsoft.com/en-us/azure/batch/batch-account-create-portal). Make a note of the ***account name***, primary ***access key*** and the ***batch account URL*** as you will need those shortly as well. You can find this information in the properties section of your batch account on the Azure portal.

## Fill in the network configuration file

In the tutorial repository, you will find a file called **network_config.json** with some empty fields. The rest of this tutorial will use this file to access your account information for the different Azure services used. Please follow these guidelines to fill in your information:

* **"subscription_id"**: This is your Azure subscription ID which will be charged for the resources you use
* **"resource_group_name"**: This is the name of the resource group you created your storage account in (recorded above)
* **"storage_account_name"**: This is the storage account name recorded above
* **"storage_account_key"**: This is the primary access key to your storage account recorded above
* **"file_share_name"**: Choose a name for your file share
* **"batch_account_name"**: This is the name of your Batch account recorded above
* **"batch_account_key"**: This is the primary access key to your Batch account recorded above
* **"batch_account_url"**: This is the batch account URL recorded above
* **"batch_job_user_name"**:  Choose a username
* **"batch_job_user_password"**: Choose a password
* **"batch_pool_name"**: Choose a name for your pool of machines
* **"batch_pool_size"**: The total number of virtual machines you want to use in your pool (minimum 2). You will need one machine to act as the parameter server and rest will take on the role of agents. For example, if you want to distribute training across 5 agent VMs, you will use a batch pool size of 6. 

Before setting up the cluster, you need to set up an Azure File Share to host the executable and the script files. Let's begin by importing some prerequisite libraries. 

In [1]:
#Standard python libraries
import json
import os
import re
import datetime
import time

from IPython.display import clear_output

#Azure file storage. To install, run 'pip install azure-storage-file'
from azure.storage.file import FileService
from azure.storage.file import ContentSettings

#Azure blob. To install, run 'pip install azure-storage-blob'
from azure.storage.blob import BlockBlobService
from azure.storage.blob import PublicAccess

#Azure batch. To install, run 'pip install cryptography azure-batch azure-storage'
import azure.storage.blob as azureblob
import azure.batch.models as batchmodels
import azure.batch.batch_auth as batchauth
import azure.batch as batch

with open('notebook_config.json', 'r') as f:
    NOTEBOOK_CONFIG = json.loads(f.read()) 

Now, we will generate some prerequisite files. These files are used during the setup process to configure the virtual machines. They require information unique to your cluster which we will access from the config file you created above. The three prerquisite files that will be generated are:

* **mount.bat**: This batch file mounts an azure file share to a machine. It will mount the specified file share to the *Z:\\* directory
* **run_airsim_on_user_login.xml**: This XML file defines a scheduled task that will restart the AirSim simulator when a user logs into an agent node. This is necessary because Azure Batch starts the executable in session 0, which means that the simulator will be accessible via API, but not visible on the screen. By restarting it on login, we can visualize the training process.
* **setup_machine.py**: This script installs the prerequisite python libraries and configures the machine to properly run AirSim. 

In [6]:
#Generate mount.bat
with open('Template\\mount_bat.template', 'r') as f:
    mount_bat_cmd = f.read()
    
mount_bat_cmd = mount_bat_cmd\
                    .replace('{storage_account_name}', NOTEBOOK_CONFIG['storage_account_name'])\
                    .replace('{file_share_name}', NOTEBOOK_CONFIG['file_share_name'])\
                    .replace('{storage_account_key}', NOTEBOOK_CONFIG['storage_account_key'])

with open('Blob\\mount.bat', 'w') as f:
    f.write(mount_bat_cmd)
    
#Generate setup_machine.py
with open('Template\\setup_machine_py.template', 'r') as f:
    setup_machine_py = f.read()

setup_machine_py = setup_machine_py\
                    .replace('{storage_account_name}', NOTEBOOK_CONFIG['storage_account_name'])\
                    .replace('{file_share_name}', NOTEBOOK_CONFIG['file_share_name'])\
                    .replace('{storage_account_key}', NOTEBOOK_CONFIG['storage_account_key'])\
                    .replace('{batch_job_user_name}', NOTEBOOK_CONFIG['batch_job_user_name'])\
                    .replace('{batch_job_user_password}', NOTEBOOK_CONFIG['batch_job_user_password'])

with open('Blob\\setup_machine.py', 'w') as f:
    f.write(setup_machine_py)
    
#Generate run_airsim_on_user_login.xml
with open('Template\\run_airsim_on_user_login_xml.template', 'r', encoding='utf-16') as f:
    startup_task_xml = f.read()
    
startup_task_xml = startup_task_xml\
                    .replace('{batch_job_user_name}', NOTEBOOK_CONFIG['batch_job_user_name'])

with open('Share\\scripts_downpour\\run_airsim_on_user_login.xml', 'w', encoding='utf-16') as f:
    f.write(startup_task_xml)    

Now that we have all of the prerequisite files generated, the next step is to create the Azure File Share. We create the file share and upload all of the files under the */Share* directory of the downloaded data files. Inside this directory, there are two folders:

* **data**: This folder contains data files used by the executable. You will explore the uses of these files in [Step 1 - Explore the Algorithm](ExploreAlgorithm.ipynb).
* **scripts_downpour**: This folder contains the actual scripts that will be executed during the batch job. For more information about these scripts, see [Step 1 - Explore the Algorithm](ExploreAlgorithm.ipynb).
* **tools**: This folder contains some auxillary tools used to set up the VMs (e.g. [AzCopy](https://docs.microsoft.com/en-us/azure/machine-learning/team-data-science-process/move-data-to-azure-blob-using-azcopy) and [7zip](http://www.7-zip.org/))

In [25]:
file_service = FileService(account_name = NOTEBOOK_CONFIG['storage_account_name'], account_key=NOTEBOOK_CONFIG['storage_account_key'])
file_service.create_share(NOTEBOOK_CONFIG['file_share_name'], fail_on_exist=False)

True

Upload all files to share.

In [ ]:
def create_directories(path, file_service):
    split_dir = path.split('\\')
    for i in range(1, len(split_dir)+1, 1):
        combined_dir = '\\'.join(split_dir[:i])
        file_service.create_directory(NOTEBOOK_CONFIG['file_share_name'], combined_dir, fail_on_exist=False)

for root, directories, files in os.walk('Share'):
    for file in files:
        regex_pattern = '{0}[\\\\]?'.format('Share').replace('\\', '\\\\')
        upload_directory = re.sub(regex_pattern, '', root)
        print('Uploading {0} to {1}...'.format(os.path.join(root, file), upload_directory))
        if (len(upload_directory) == 0):
            upload_directory = None
        if (upload_directory != None):
            create_directories(upload_directory, file_service)
        file_service.create_file_from_path(          
            NOTEBOOK_CONFIG['file_share_name'], 
            upload_directory,                   
            file,                               
            os.path.join(root, file)            
            )
        
        

When provisioning the machines for the Azure Batch pool, it is necessary to pull some of the setup scripts from blob. So in this step, we upload these prerequisite files to the blob storage. 

In [12]:
block_blob_service = BlockBlobService(account_name = NOTEBOOK_CONFIG['storage_account_name'], account_key = NOTEBOOK_CONFIG['storage_account_key'])
block_blob_service.create_container('prereq', public_access = PublicAccess.Container)

for root, directories, files in os.walk('Blob'):
    for file in files:
        block_blob_service.create_blob_from_path( 
            'prereq',                             
            file,                                 
            os.path.join(root, file)              
            )

We have a custom image that has the proper drivers installed to run AirSim. To create this image, we will run a powershell script that will copy the image from our storage account to your storage account. Ensure that you have the latest version of the [AzCopy utility](https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy) installed and in your path (i.e. running 'azcopy' at the command line should yield the help page). In addition, ensure that you have the latest version of [Azure Powershell](https://docs.microsoft.com/en-us/powershell/azure/install-azurerm-ps?view=azurermps-5.3.0) installed. This command can take up to an hour to run.

In [16]:
os.system('powershell.exe ".\\CreateImage.ps1 -subscriptionId {0} -storageAccountName {1} -storageAccountKey {2} -resourceGroupName {3}'\
          .format(NOTEBOOK_CONFIG['subscription_id'], NOTEBOOK_CONFIG['storage_account_name'], NOTEBOOK_CONFIG['storage_account_key'], NOTEBOOK_CONFIG['resource_group_name']))

0

Finally, we create the pool of machines that will run our experiment. The important aspects of the machine configuration are:

* **image_reference**: We specify the data science VM to ensure that we have the correct drivers installed that will allow us to utilize the GPU.
* **vm_size**: The AirSim executable will only run on NV-series virtual machines, so we choose the NV6 VM SKU for this tutorial. (You can later change this in *Template/pool.json.template*. Please make sure you choose an NV-series VM if you do this.)
* **target_dedicated_nodes**: The number of nodes to provision for the cluster. Note that 1 node will become your trainer, and the rest will become the agent. Ensure that there are enough cores available in your batch account to provision the number of VMs you are requesting - for example, the NV6 machines utilize 4 cores for each machine provisioned.
* **enable_inter_node_communication**: This parameter will allow the nodes to communicate with each other. Enabling this parameter limits the number of nodes to 40.
* **user_accounts**: We define an admin user to run the batch jobs. This user will also be used to log into the VMs and visualize the progress
* **start_task**: This is the task that will be run when the machines are being provisioned. In this phase, we download the prereq scripts and run them in the python environment. This will install the necessary python libraries and configure the machine to use AirSim.

We will use the azure CLI to deploy the cluster. The complete configuration used for the cluster can be seen in the generated pool.json file. Note that to use the CLI, you will need to manually authenticate. Check the terminal window for authentication instructions when running this code segment.

In [ ]:
with open('Template\\pool.json.template', 'r') as f:
    pool_config = f.read()
    
pool_config = pool_config\
                .replace('{batch_pool_name}', NOTEBOOK_CONFIG['batch_pool_name'])\
                .replace('{subscription_id}', NOTEBOOK_CONFIG['subscription_id'])\
                .replace('{resource_group_name}', NOTEBOOK_CONFIG['resource_group_name'])\
                .replace('{storage_account_name}', NOTEBOOK_CONFIG['storage_account_name'])\
                .replace('{batch_job_user_name}', NOTEBOOK_CONFIG['batch_job_user_name'])\
                .replace('{batch_job_user_password}', NOTEBOOK_CONFIG['batch_job_user_password'])\
                .replace('{batch_pool_size}', str(NOTEBOOK_CONFIG['batch_pool_size']))

with open('pool.json', 'w') as f:
    f.write(pool_config)
    
create_cmd = 'powershell.exe ".\ProvisionCluster.ps1 -subscriptionId {0} -resourceGroupName {1} -batchAccountName {2}"'\
    .format(NOTEBOOK_CONFIG['subscription_id'], NOTEBOOK_CONFIG['resource_group_name'], NOTEBOOK_CONFIG['batch_account_name'])
    
print('Executing command. Check the terminal output for authentication instructions.')

os.system(create_cmd)

Once this task finishes, you should see the pool created in your Batch account, and you are ready to move on to **[Step 1 - Explore the Algorithm](ExploreAlgorithm.ipynb)** 